In [1]:
import pandas as pd
import os
import re

In [2]:
import os
os.getcwd() 
os.chdir('/Users/hannahz/Desktop/G5055_Practicum_Project2/Data/Text_Model_Data')

In [3]:
# list all the pdfs
pdf_path = os.listdir('SDG-indicator-pdfs')

#add in directory
pdf_path = ['SDG-indicator-pdfs/' + path for path in pdf_path]

# there's one called .ds-store, dont know why, so i deleted it'
#pdf_path.remove('SDG-indicator-pdfs/.DS_Store')


In [4]:
len(pdf_path)

246

In [15]:
#install package
#pip install PyMuPDF

In [5]:
import fitz

def get_text(filepath: str) -> str:
    with fitz.open(filepath) as doc:
        text = ""
        for page in doc:
            text += page.getText()
        return text


In [177]:
import pandas as pd 
lst1 = pdf_path
lst2 = [get_text(path) for path in pdf_path]

percentile_list = pd.DataFrame(
    {'pdf_name': lst1,
     'pdf_text': lst2
    })
percentile_list = percentile_list.sort_values('pdf_name')
percentile_list.to_csv('text_chunk.csv')

In [3]:
text_chunk = pd.read_csv('text_chunk.csv')
text_chunk

,Unnamed: 0,pdf_name,pdf_text
0,217,SDG-indicator-pdfs/Metadata-01-01-01a.pdf,Last updated: July 2021 \nSDG indicator metada...
1,193,SDG-indicator-pdfs/Metadata-01-02-01.pdf,Last updated: August 2021 \nSDG indicator meta...
2,205,SDG-indicator-pdfs/Metadata-01-02-02.pdf,Last updated: May 2021 \nSDG indicator metadat...
3,114,SDG-indicator-pdfs/Metadata-01-03-01a.pdf,Last updated: January 2021 \nSDG indicator met...
4,140,SDG-indicator-pdfs/Metadata-01-04-01.pdf,Last updated: March 2021 \nSDG indicator metad...
...,...,...,...
241,115,SDG-indicator-pdfs/Metadata-17-17-01.pdf,Last updated: September 2020 \nGoal 17: Streng...
242,55,SDG-indicator-pdfs/Metadata-17-18-02.pdf,Last updated: 13 February 2018 \n \nGoal 17: S...
243,50,SDG-indicator-pdfs/Metadata-17-18-03.pdf,Last updated: 11 July 2017 \n \nGoal 17: Stren...
244,165,SDG-indicator-pdfs/Metadata-17-19-01.pdf,Last updated: 11 July 2017 \n \nGoal 17: Stren...


In [58]:
goal_target_indicator = pd.read_excel('Goal_Target_Indicator.xlsx')
goal_target_indicator.head()

,Index,Goal,Target,Indicator
0,1.1.1,Goal 1. End poverty in all its forms everywhere,"1.1 By 2030, eradicate extreme poverty for all...",1.1.1 Proportion of the population living belo...
1,1.2.1,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...",1.2.1 Proportion of population living below th...
2,1.2.2,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...","1.2.2 Proportion of men, women and children of..."
3,1.3.1,Goal 1. End poverty in all its forms everywhere,1.3 Implement nationally appropriate social pr...,1.3.1 Proportion of population covered by soci...
4,1.4.1,Goal 1. End poverty in all its forms everywhere,"1.4 By 2030, ensure that all men and women, in...",1.4.1 Proportion of population living in house...


In [59]:
# find the definition from .pdf files

definition_all = {}
d_pattern = 'Definition:'
c_pattern = 'Concepts:'
error = []
for i in range(text_chunk.shape[0]):
    try:
        test_str = text_chunk.iloc[i]['pdf_text']
        definition_index = [(m.start(0), m.end(0)) for m in re.finditer(d_pattern, test_str)]
        concept_goal = [(m.start(0), m.end(0)) for m in re.finditer(c_pattern, test_str)]
        definition = test_str[definition_index[0][1]+2:concept_goal[0][0]]
        definition_all[text_chunk.iloc[i]['pdf_name']] = definition
    except:
        error.append(text_chunk.iloc[i]['pdf_name'])
        #print(text_chunk.iloc[i]['pdf_name'])
        definition_all[text_chunk.iloc[i]['pdf_name']] = ''

text_chunk['definition']= text_chunk['pdf_name'].map(definition_all)
df = pd.DataFrame(definition_all.items())

In [60]:
#keep XX-XX-XXa eg 01-01-01
df['Index'] = df[0].apply(lambda x: x.replace('SDG-indicator-pdfs/Metadata-',"").replace('.pdf',''))
df = df.rename(columns={0:'pdf_name',1:'definition' })

In [61]:
#change 01 02 to 1 2 eg 01-01-01 to 1-01-01
df['Index'] = df['Index'].apply(lambda x: x[1:] if x[0] == '0' else x)
#lowercase the string
df['Index'] = df["Index"]. str. lower() 
#replace -0 with .
df['Index'] = df['Index'].apply(lambda x: x.replace('-0',".").replace('-','.'))
#get rid of the end 'a' eg 1.1.1a to 1.1.1
df['Index'] = df['Index'].apply(lambda x: x.replace('a','') if x[-1] == 'a' else x)
#get rid of the end ' ' 
goal_target_indicator['Index'] = goal_target_indicator['Index'].apply(lambda x: str(x).strip())

In [64]:
goal_target_indicator_definition = goal_target_indicator.merge(df, on = 'Index', how = 'left')
goal_target_indicator_definition.to_csv('goal_target_indicator_definition.csv')

In [63]:
goal_target_indicator_definition.head()

,Index,Goal,Target,Indicator,pdf_name,definition
0,1.1.1,Goal 1. End poverty in all its forms everywhere,"1.1 By 2030, eradicate extreme poverty for all...",1.1.1 Proportion of the population living belo...,SDG-indicator-pdfs/Metadata-01-01-01a.pdf,The indicator “proportion of the population be...
1,1.2.1,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...",1.2.1 Proportion of population living below th...,SDG-indicator-pdfs/Metadata-01-02-01.pdf,The national poverty rate is the percentage of...
2,1.2.2,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the propo...","1.2.2 Proportion of men, women and children of...",SDG-indicator-pdfs/Metadata-01-02-02.pdf,The following four series are used to monitor ...
3,1.3.1,Goal 1. End poverty in all its forms everywhere,1.3 Implement nationally appropriate social pr...,1.3.1 Proportion of population covered by soci...,SDG-indicator-pdfs/Metadata-01-03-01a.pdf,The indicator reflects the proportion of perso...
4,1.4.1,Goal 1. End poverty in all its forms everywhere,"1.4 By 2030, ensure that all men and women, in...",1.4.1 Proportion of population living in house...,SDG-indicator-pdfs/Metadata-01-04-01.pdf,
...,...,...,...,...,...,...
241,17.17.1,Goal 17. Strengthen the means of implementatio...,"17.17 Encourage and promote effective public, ...",17.17.1 Amount in United States dollars commit...,SDG-indicator-pdfs/Metadata-17-17-01.pdf,Indicator based on WBG data: “Amount of United...
242,17.18.2,Goal 17. Strengthen the means of implementatio...,17.18.2 Number of countries that have national...,17.18.2 Number of countries that have national...,SDG-indicator-pdfs/Metadata-17-18-02.pdf,The indicator refers to the number of countrie...
243,17.18.3,Goal 17. Strengthen the means of implementatio...,17.18.3 Number of countries with a national st...,17.18.3 Number of countries with a national st...,SDG-indicator-pdfs/Metadata-17-18-03.pdf,
244,17.19.1,Goal 17. Strengthen the means of implementatio...,"17.19 By 2030, build on existing initiatives t...",17.19.1 Dollar value of all resources made ava...,SDG-indicator-pdfs/Metadata-17-19-01.pdf,
